<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/Fault_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Navigate to project folder and ensure we're on the 'develop' branch
import os, subprocess, sys

REPO_DIR = "/content/Cloud_Computing"
if not os.path.isdir(REPO_DIR):
    subprocess.run([
        "git", "clone", "-b", "develop",
        "https://github.com/Method-for-Software-System-Development/Cloud_Computing.git",
        REPO_DIR
    ], check=True)

# Update & checkout develop if repo already exists
subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "checkout", "develop"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

# Install importnb (only needed once per session)
%pip install importnb --quiet

# Add firebase folder to Python path and import its notebook as module
sys.path.append(os.path.join(REPO_DIR, "firebase"))  # for FireBase.ipynb
sys.path.append(REPO_DIR)                            # for FAULT_RULES.ipynb
from importnb import Notebook

with Notebook():
    import FireBase as fb                           # now fb.add_user, fb.get_user, etc. are available
    from FAULT_RULES import FAULT_RULES as fr       # now FAULT_RULES are aviable to use


firebase_url = fb.firebase_url

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
omer_cohen 200
shira_gold 200
daniel_bar 200
roni_benami 200
lior_levi 200
yael_elyashiv 200
noa_katz 200
matan_tal 200
or_peled 200
tom_segal 200
Indoor Sensor Data:
Status: 200 | Uploaded: {'Temperature': 26.4, 'Humidity': 41.0, 'Pressure': 974.1, 'Distance': 210.1}
Status: 200 | Uploaded: {'Temperature': 30.2, 'Humidity': 28.7, 'Pressure': 963.5, 'Distance': 8.9}
Status: 200 | Uploaded: {'Temperature': 36.5, 'Humidity': 91.3, 'Pressure': 919.8, 'Distance': 12.2}
Status: 200 | Uploaded: {'Temperature': 27.1, 'Humidity': 73.0, 'Pressure': 1050.4, 'Distance': 205.0}
Status: 200 | Uploaded: {'Temperature': 25.8, 'Humidity': 45.2, 'Pressure': 960.0, 'Distance': 210.0}
Status: 200 | Uploaded: {'Temperature': 32.7, 'Humidity': 29.5, 'Pressure': 940.2, 'Distance': 7.5}
Status: 200 | Uploaded: {'Temperature': 28.9, 'Humidity': 89.9, 'Pressure': 950.6, 'Distance': 250.0}
Status: 200 | Uploaded: {'Temperature': 30.0,

In [ ]:
import requests
import json
from datetime import datetime

def process_faults_from_database():
    active_faults = fb.get_active_faults()
    existing_titles = set()

    if isinstance(active_faults, dict):
        existing_titles = {
            fault["title"]
            for fault in active_faults.values()
            if isinstance(fault, dict) and "title" in fault
        }

    location_map = {
        "IndoorData": "Indoor",
        "OutdoorData": "Outdoor"
    }

    for folder, location in location_map.items():
        url = f"{firebase_url}SensorsData/{folder}.json"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to load {folder}")
            continue

        data = response.json()
        if not isinstance(data, dict):
            print(f"No valid data in {folder}")
            continue

        for reading in data.values():
            if not isinstance(reading, dict):
                continue

            for sensor_type, value in reading.items():
                key = f"{sensor_type.lower()}_{location.lower()}"
                for rule in fr.get(key, []):
                    min_val = rule.get("min")
                    max_val = rule.get("max")

                    if (min_val is None or value >= min_val) and (max_val is None or value < max_val):
                        if rule["title"] in existing_titles:
                            continue

                        title_key = f"{rule['title']} ({datetime.now().strftime('%d-%m-%Y %H:%M:%S')})"

                        fault = {
                            "sensor": rule["sensor"],
                            "severity": rule["severity"],
                            "title": rule["title"],
                            "value": value,
                            "status": "Active",
                            "repaired_by": "",
                            "timestamp": datetime.now().isoformat(),
                            "actions": rule["actions"]
                        }

                        fb.add_active_fault(title_key, fault)
                        existing_titles.add(rule["title"])
                        print(f"[{location}] Fault Created: {rule['title']} (Severity: {rule['severity']})")

process_faults_from_database()